In [ ]:
# Commands to run in the terminal to install the required packages
""" 
conda create -n ollama python=3.10.9
conda activate ollama
pip install -r requirements.txt
"""
# ! pip install ollama 

In [4]:
# Imports
import os
import pandas as pd
import ollama
import chromadb
#from langchain_chroma import Chroma
#from langchain_community.embeddings import OllamaEmbeddings
#import chromadb.utils.embedding_functions as embedding_functions
from chromadb.config import Settings

# Constants Definition
PERSIST_DIRECTORY = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\datasets_db"


In [ ]:
# Lets see how many model are included
import ollama

# Pull mistral model
# ollama.pull("mistral")
# ollama.show("mistral")
ollama.list()['models']

## Test with "ollama2"

In [ ]:
# Set model to llama2 and chat with it
stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'What is Flagger Force?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

## Add Context to the Model Response


In [ ]:
import ollama
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = WebBaseLoader(    
    web_paths=("https://flaggerforce.com/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("page-content", "page-body", "page-header")
        )
    ),
)
docs = loader.load()
docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits

In [ ]:
docs

## 2.1

In [ ]:
# Import users file
import os
import pandas as pd

#path = "/home/ubuntu/thesis_GenAI/data/production_datasets/cleaned_datasets"
path = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\production_datasets\\cleaned_datasets"
file_name = "users_cleaned.csv"
file_path = os.path.join(path, file_name)
df_users = pd.read_csv(file_path)
df_users.head()

# df_users.iloc[:20,:].to_csv("C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\production_datasets\\cleaned_datasets\\users_to_test.csv")

In [ ]:
import csv
with open ("C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\production_datasets\\cleaned_datasets\\users_to_test.csv") as file:
    lines = csv.reader(file)
    
    # Initialize list variables
    documents = []
    metadatas = []
    ids = []
    id = 1
    
    # Iterate through csv by row
    for i, line in enumerate(lines):
        if i==0:
            continue
        
        documents.append(line[3])   # Adding the username as main information
        metadatas.append({'first_login':line[1], 'id':line[4], 'position':line[17], 'department':line[15], 'suspended':line[2], 
                          'last_skipped_contact_details':line[5], 'state':line[6], 'role':line[7], 'last_login':line[8],'updated':line[9],
                          'used':line[10], 'display_name_extension':line[11], 'suspended_at':line[12], 'language':line[13], 'created':line[14], 'location':line[16]})
        ids.append(str(id))
        id+=1    

In [ ]:
import chromadb

chroma_client = chromadb.Client(ALLOW_RESET=TRUE)
collection_users = chroma_client.create_collection(name="users_collection")
# collection_posts = chroma_client.create_collection(name="posts_collection")
# collection_comments = chroma_client.create_collection(name="comments_collection")
# collection_streams = chroma_client.create_collection(name="streams_collection")

In [ ]:
#chroma_client.delete_collection(name="users_collection")
#vectorstore.delete_collection()

chroma_client.reset()

In [ ]:
"""
Chroma uses any Sentence Transformers (https://www.sbert.net/) model to create embeddings. Default model is "all-MiniLM-L6-v2",
other model are available: https://www.sbert.net/docs/pretrained_models.html  
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
"""
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

PERSIST_DIRECTORY = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\vectordb"
embedding = OllamaEmbeddings(model="mistral")
vectorstore = Chroma.from_documents(    
    documents=splits, 
    embedding=embedding,
    #metadatas=metadatas,
    ids=ids,
    persist_directory=PERSIST_DIRECTORY
    )


# collection_users.add(
#     #embeddings=[[1.2, 2.3, 4.5], [6.7, 8.2, 9.2]],
#     documents=documents,
#     embeddings=embeddings,
#     metadatas=metadatas,
#     ids=ids)

In [ ]:
results = collection_users.query(
    query_texts=["Aaron"],
    n_results=2,
    include=['distances','documents']
)

results

## Adding Comments to Chromadb

In [2]:
# Import users file
import os
import pandas as pd

#path = "/home/ubuntu/thesis_GenAI/data/production_datasets/cleaned_datasets"
path = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\production_datasets\\cleaned_datasets"
file_name = "comments_cleaned.csv"
file_path = os.path.join(path, file_name)
df_comments = pd.read_csv(file_path)

# Create a sample of 50,000 comments to test
df_comments_to_test = df_comments.sample(300)
# Print a sample
df_comments.sample(5)

,stream_name,created_date,post_id,comment_id,comment_text,like_count,report_count,username,author_user_id,author_position,author_status
26493,Jobs,2023-09-22 05:20:45,7050379,6232334,what happened?,0,0,Vera_Rivera,46a5d6ae-52ae-4b13-8add-65e46ed7d98a,Crew Member,active
33229,Jobs,2023-08-13 12:20:27,6915828,6078965,I will go,0,0,Ira_Crusan,0826bcb2-a311-443a-a3a9-d62c169388a7,Crew Leader,suspended
92731,Safety & Operations,2022-09-13 17:59:39,5751264,4800765,@Amber_Davenport congratulations! Thank you fo...,1,0,Scott_Richwine,2aac35e8-f970-4434-9b46-56274384cf89,Safety Manager,active
41111,Water Break,2023-06-29 23:40:53,6753487,5912239,DOSE ANYONE KNOW WERE TO GET THIS,0,0,David_Unkelbach,47aa9966-5f04-47c3-b90b-753aa742753f,Advanced Crew Leader,active
73580,Jobs,2022-12-24 14:04:15,6106979,5194273,Ty,1,0,Charles_Yates,03bea137-fadd-40a5-af8c-52c93922da53,Advanced Crew Leader,active


In [23]:
# count how times a user has commented
df_comments_to_test['author_position'].value_counts().to_frame()

,count
author_position,
Advanced Crew Leader,207
Crew Member,109
Crew Leader,90
Weekend Dispatch,10
Lead Instructor,10
Area Supervisor,8
Field Trainer 1,6
Warehouse Coordinator,5
Field Trainer 2,4


In [23]:
df_comments[df_comments['username']=='FlaggerForce']

,stream_name,created_date,post_id,comment_id,comment_text,like_count,report_count,username,author_user_id,author_position,author_status
0,Water Break,2024-03-05 15:15:55,7612625,6866587,Congratulations and a great addition to the te...,0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
1,Water Break,2024-03-05 15:10:47,7612663,6866567,"Hello team, remember to have safety discussion...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
2,Water Break,2024-03-05 15:08:29,7613861,6866561,"@Larry_Broderick, both our moderation team and...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
3,Safety & Operations,2024-03-05 15:04:42,7613237,6866545,"@William_Pulliam, we appreciate you being prou...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
4,Safety & Operations,2024-03-05 15:02:47,7613255,6866539,"We appreciate you supporting @Dennis_Cumbie, @...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
...,...,...,...,...,...,...,...,...,...,...,...
139662,Safety & Operations,2022-01-03 21:28:23,4928202,3861062,Big kudos to @Karen_Skwara.She always goes abo...,2,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
139664,Safety & Operations,2022-01-03 21:25:24,4929313,3861053,First snow of the season @April_Long?,0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
139665,Safety & Operations,2022-01-03 21:24:39,4929597,3861047,Excellent safety message @Samuel_Wirtz. Clear ...,1,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
139667,Safety & Operations,2022-01-03 21:22:24,4930738,3861033,We are very grateful for this good news. Stay ...,0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active


In [3]:
# Function to prepare the data to be loaded to the database
def prepare_data(df):   
    documents = []
    metadatas = []
    ids = []
        
    for idx, row in df.iterrows(): 
        documents.append(row.iloc[4])
        metadatas.append({
            'stream_name': row.iloc[0],
            'created_date': row.iloc[1],
            'post_id': row.iloc[2],
            'comment_id': row.iloc[3],
            'like_count': row.iloc[5],
            'report_count': row.iloc[6],
            'username': row.iloc[7],
            'author_user_id': row.iloc[8],
            'author_position': row.iloc[9],
            'author_status': row.iloc[10]
        })
        ids.append(str(idx+1))       
    
    return documents, metadatas, ids

documents, metadatas, ids = prepare_data(df_comments_to_test)

print(type(documents))
documents

<class 'list'>


['@Harold_Friscia lol. In my backyard lol.',
 'Congratulations Austin!!',
 '@Duwan_Maye oh they playing with you bruh lol',
 '@Julie_Snedeker when I left from up there it was raining ice, by the time I got back in MD it was raining.',
 'Me for Baltimore location @weekend_dispatch @weekend_dispatch',
 'Available',
 "I called your stand in for you. She got it canceled, but the TCI training wasn't  canceled. That's why I got the 2 points. I knew you were away doing training, etc. That's why I called your backup. I met her today also. Don't  remember her name but she came from Georgia, I believe.",
 'My condolences to his family 🕊️',
 'With you',
 'Welcome to Flagger Force!',
 'Okay, cat lady! 🤣',
 'Me too',
 'should be interesting',
 'I can go',
 '@Raymond_Moody and @Edwin_SkinnerJr  you have been scheduled.  Thank you!!!!',
 'i am',
 'Congrts u two well done',
 '@jason_miller  no problem. I wasn’t sure where in western you were.',
 '@James_WootenIII and me',
 "Also for number 1, it depen

In [40]:
metadatas[0]

{'stream_name': 'Safety & Operations',
 'created_date': '2022-12-28 13:18:45',
 'post_id': 6113990,
 'comment_id': 5205991,
 'like_count': 3,
 'report_count': 0,
 'username': 'FlaggerForce',
 'author_user_id': 'b701ab9f-563a-4425-a389-aff803a8da58',
 'author_position': nan,
 'author_status': 'active'}

In [11]:
import chromadb
#from langchain_chroma import Chroma
#from langchain_community.embeddings import OllamaEmbeddings
#import chromadb.utils.embedding_functions as embedding_functions
from chromadb.config import Settings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
      
PERSIST_DIRECTORY = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\datasets_db"
embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")    
    

# Function to create and load data to the database
"""
The embedding function takes text as input, and performs tokenization and embedding. If no embedding function is supplied, Chroma will use sentence transformer as a default.
https://docs.trychroma.com/embeddings#sentence-transformers
By default, Chroma uses all-MiniLM-L6-v2
"""


def upload_to_chromadb(documents, metadatas, ids):
    try:
        # Initialize ChromaDB client
        #chromadb_client = chromadb.EphemeralClient()      
        chromadb_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY, settings=Settings(allow_reset=True))
        
        # create the open-source embedding function
        # embedding = OllamaEmbeddings(model="mistral")
        embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")   # Default
        embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")          # Best model from included in ChromaDB
               
        # Create a comment's collection
        collection_comments = chromadb_client.create_collection(name="comments_collection",
                                                              metadata={"hnsw:space": "l2"}, # Squared L2 norm(l2) is the default, inner product('ip') or cosine similarity('cosine')                                                              
                                                              #embedding_function=embeddings
                                                              )        
        
        # Add data to collection
        collection_comments.add(documents=documents,
                                metadatas=metadatas,
                                ids=ids
                                )
        print("Data uploaded successfully.")
    except Exception as e:
        print(f"Failed to upload data: {e}")

upload_to_chromadb(documents, metadatas, ids)

Data uploaded successfully.


In [12]:
# Get the first 10 documents in comments_collection
# Initialize database
PERSIST_DIRECTORY = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\datasets_db"
chromadb_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY, settings=Settings(allow_reset=True))
comments_collection = chromadb_client.get_collection(name="comments_collection")
chromadb_client.heartbeat()


# Only get documents and ids
result = comments_collection.get()
print(len(result))
result[:10]

4


TypeError: unhashable type: 'slice'

In [27]:
result.keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])

In [13]:
# Deleting the client
import chromadb
from chromadb.config import Settings
PERSIST_DIRECTORY = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\datasets_db"

chromadb_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY, settings=Settings(allow_reset=True))
chromadb_client.reset()

True

In [6]:
# Delete the collection
chromadb_client.delete_collection(name="comments_collection")

In [66]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import OllamaEmbeddings
# Use the query method to search for similar comments
text = "We appreciate your support"
#username = 'Ebony_Scott'
#username = 'John_Ware'    # User with many comments
username = 'FlaggerForce'
num_comments = 3
min_num_likes = 3

# Define a function to query the database
def query_database(text, username, num_comments=3, min_num_likes=3):
    try:
        # Initialize database
        chromadb_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY, settings=Settings(allow_reset=True))
        comments_collection = chromadb_client.get_collection(name="comments_collection")
        
        # Query the database
        context_docs = comments_collection.query(
            query_texts=[text],
            n_results=num_comments,
            include=['distances','documents'],
            #where={"$and": [{"username": username}, {"like_count": {"$gt": min_num_likes}}]}
            where={"like_count": {"$gt": min_num_likes}}
        )
        return context_docs
    except Exception as e:
        print(f"Failed to query the database: {e}")
        
# Query the database
retrieved_docs = query_database(text, username, num_comments, min_num_likes)
retrieved_docs

{'ids': [['82239', '47464', '129219']],
 'distances': [[1.0975472927093506, 1.3787870407104492, 1.386555552482605]],
 'metadatas': None,
 'embeddings': None,
 'documents': [['Very well written post!!!!!  I absolutely love this. ❤ I am truly thankful for the wonderful teamwork that is displayed every day and for being a part of the FF Family',
   'What a beautiful set up!!! GREAT JOB everyone!!!!!',
   'Congratulations @Roniece_McFadden keep going on up that ladder. You are going to do great out there.']],
 'uris': None,
 'data': None}

In [63]:
retrieved_docs['documents']

[['Very well written post!!!!!  I absolutely love this. ❤ I am truly thankful for the wonderful teamwork that is displayed every day and for being a part of the FF Family',
  'What a beautiful set up!!! GREAT JOB everyone!!!!!',
  'Congratulations @Roniece_McFadden keep going on up that ladder. You are going to do great out there.']]

In [67]:
# Split documents into chunks
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb

# Define a function to split and format the documents
def split_documents(documents):
    text_splitter = CharacterTextSplitter(
        separator=".",
        chunk_size=100, 
        chunk_overlap=0)
    splits = text_splitter.create_documents(documents['documents'][0])
    return splits

splitted_docs = split_documents(retrieved_docs)
splitted_docs


# Function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

formatted_context = format_docs(splitted_docs)
formatted_context
# # create the open-source embedding function
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# langchain_chroma = Chroma(
#     client=chromadb_client,
#     collection_name="comments_collection",
#     embedding_function=embedding_function,
# )

'Very well written post!!!!!  I absolutely love this\n\n❤ I am truly thankful for the wonderful teamwork that is displayed every day and for being a part of the FF Family\n\nWhat a beautiful set up!!! GREAT JOB everyone!!!!!\n\nCongratulations @Roniece_McFadden keep going on up that ladder. You are going to do great out there'

In [70]:
# Imports
import ollama
from langchain_community.embeddings import OllamaEmbeddings

# Function that defines the RAG chain
def rag_chain(question):
    retrieved_docs = query_database(question, username, num_comments, min_num_likes)
    splitted_docs = split_documents(retrieved_docs)
    formatted_prompt = format_docs(splitted_docs)
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Test the RAG chain
question = "Make a welcome post for Daniel Meier who has just started a new position as Crew Leader"
response = rag_chain(question)
response

Created a chunk of size 159, which is longer than the specified 100
Created a chunk of size 150, which is longer than the specified 100


" Thank you for your kind words and appreciation, Corey! I'm glad that our teamwork and collaboration have made a positive impact on your experience at FF. It's always a pleasure to work together and support each other in our daily tasks. Your story about the light towers and bathroom break is a great example of how we prioritize safety and teamwork, even during challenging situations.\n\nI'm also glad that you found our Pittsburgh-style welcoming and decided to give social media a try! We're excited to have you as part of our team, and always prioritize safety above all else. Let's continue working together to make FF the best it can be! Go Steelers! 😉🏈💪🏼 #FFFamily #TeamworkMakesTheDreamWork"

In [8]:
# Initialize database
chromadb_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY, settings=Settings(allow_reset=True))
comments_collection = chromadb_client.get_collection(name="comments_collection")

# Use the query method to search for similar comments
text = "We appreciate your support"
#username = 'Ebony_Scott'
#username = 'John_Ware'    # User with many comments
username = 'FlaggerForce'
num_comments = 3
min_num_likes = 3

# Query the database
context_docs = comments_collection.query(    
    query_texts=[text],
    n_results=num_comments,
    include=['distances','documents'],
    #where={"$and": [{"username": username}, {"like_count": {"$gt": min_num_likes}}]}
    where={"like_count": {"$gt": min_num_likes}}
)

context_docs

{'ids': [['82239', '47464', '129219']],
 'distances': [[1.0975472927093506, 1.3787870407104492, 1.386555552482605]],
 'metadatas': None,
 'embeddings': None,
 'documents': [['Very well written post!!!!!  I absolutely love this. ❤ I am truly thankful for the wonderful teamwork that is displayed every day and for being a part of the FF Family',
   'What a beautiful set up!!! GREAT JOB everyone!!!!!',
   'Congratulations @Roniece_McFadden keep going on up that ladder. You are going to do great out there.']],
 'uris': None,
 'data': None}

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

# text = """
# I think it is about time to find a more interesting job off the road. I always wanted to switch to a position with a higher administrative load of work, what do you think about it?

# Text models specially those centered around conversational chatbots have revolutionized AI since the launch of ChatGPT. Helped by natural language processing and large language models, these models have many very useful capabilities like summarization, writing assistance, code generation, language translation and sentiment analysis. 

# They have been the main focus in Generative AI because of the capabilities of ChatGPT, a application which millions of users are already taking advantage of.
# """



# Initialize database
chromadb_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY, settings=Settings(allow_reset=True))
comments_collection = chromadb_client.get_collection(name="comments_collection")

# Use the query method to search for similar comments
text = "job"
#username = 'Ebony_Scott'
username = 'Aaron_Burch'    # Most popular user, Advanced Crew Leader
num_comments = 5
min_num_likes = 1
context_docs = comments_collection.query(query_texts=[text],
                                      n_results=num_comments,
                                      include=['distances','documents'],
                                      where={"username": username, 
                                             "like_count": {"$gt": min_num_likes}
                                             },
                                      #where_document={"$contains":"congratulations"},
                                      )

context_docs

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# splits = text_splitter.create_documents([text])
# splits

# comments_collection.query(
#     query_texts=["doc10", "thus spake zarathustra", ...],
#     n_results=10,
#     where={"metadata_field": "is_equal_to_this"},
#     where_document={"$contains":"search_string"}
# )

In [ ]:
# Now we can load the persisted database from disk, and use it as normal. 
from langchain.vectorstores import Chroma
vectordb = Chroma(persist_directory=PERSIST_DIRECTORY)
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("find a new job")

In [ ]:
# Initialize database
chroma_client = chromadb.PersistentClient(path="C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\vectordb")
collection_comments = chroma_client.get_collection(name="comments_collection")
# print(collection_comments)

results = collection_comments.query(    
    query_texts=["find a job"],
    n_results=3,
    include=['distances','documents']
)

results

In [ ]:
import chromadb

# Initialize database
chroma_client = chromadb.PersistentClient(path="C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\vectordb")
chroma_client.delete_collection(name="comments_collection")
# chroma_client.reset() # Empties and completely resets the database.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

text = """
I think it is about time to find a more interesting job off the road. I always wanted to switch to a position with a higher administrative load of work, what do you think about it?

Text models specially those centered around conversational chatbots have revolutionized AI since the launch of ChatGPT. Helped by natural language processing and large language models, these models have many very useful capabilities like summarization, writing assistance, code generation, language translation and sentiment analysis. 

They have been the main focus in Generative AI because of the capabilities of ChatGPT, a application which millions of users are already taking advantage of.
"""
documents = collection_comments.query(query_texts=[text],
                                      n_results=3,
                                      include=['distances','documents'])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
splits = text_splitter.create_documents([text])
splits

embeddings = OllamaEmbeddings(model="mistral")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


In [ ]:
# Function to load and retrieve documents from ChromaDB
def load_and_retrieve_docs(query):
    # Retrieve documents based on a simple text query or other metadata-based retrieval
    documents = collection_comments.query(query)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents([doc['content'] for doc in documents])
    embeddings = OllamaEmbeddings(model="mistral")
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    return vectorstore.as_retriever()

### Test with lang chain


In [29]:
import chromadb
from chromadb.config import Settings
PERSIST_DIRECTORY = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\datasets_db"

# Initialize database
PERSIST_DIRECTORY = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\datasets_db"
chromadb_client = chromadb.PersistentClient(path=PERSIST_DIRECTORY, settings=Settings(allow_reset=True))
comments_collection = chromadb_client.get_collection(name="comments_collection")
chromadb_client.heartbeat()

1714979865447440200

In [30]:
# Lets see how many model are included
import ollama
#ollama.list()['models']

# Pull mistral model
ollama.pull("mistral")
ollama.show("mistral")


{'license': '                                 Apache License\n                           Version 2.0, January 2004\n                        http://www.apache.org/licenses/\n\n   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION\n\n   1. Definitions.\n\n      "License" shall mean the terms and conditions for use, reproduction,\n      and distribution as defined by Sections 1 through 9 of this document.\n\n      "Licensor" shall mean the copyright owner or entity authorized by\n      the copyright owner that is granting the License.\n\n      "Legal Entity" shall mean the union of the acting entity and all\n      other entities that control, are controlled by, or are under common\n      control with that entity. For the purposes of this definition,\n      "control" means (i) the power, direct or indirect, to cause the\n      direction or management of such entity, whether by contract or\n      otherwise, or (ii) ownership of fifty percent (50%) or more of the\n      outstanding

In [32]:
metadatas[0]

{'stream_name': 'Safety & Operations',
 'created_date': '2022-12-28 13:18:45',
 'post_id': 6113990,
 'comment_id': 5205991,
 'like_count': 3,
 'report_count': 0,
 'username': 'FlaggerForce',
 'author_user_id': 'b701ab9f-563a-4425-a389-aff803a8da58',
 'author_position': nan,
 'author_status': 'active'}

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI



In [35]:
from langchain_community.vectorstores import FAISS
#from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

db = Chroma(persist_directory=PERSIST_DIRECTORY)
persistent_client = chromadb.PersistentClient()


retriever = db.as_retriever()

ValueError: An instance of Chroma already exists for C:\Users\eduar\Documents\Master_Thesis\GenAI_Thesis_Beekeeper\data\datasets_db with different settings

In [ ]:
# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="mistral")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Create the retriever
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

# Use the RAG chain
result = rag_chain("What is Flagger Force?")
print(result)

## Scrapping

In [ ]:
# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="mistral")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Create the retriever
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the Ollama LLM function
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

# Use the RAG chain
result = rag_chain("What is Flagger Force?")
print(result)

In [ ]:
# Use the RAG chain
result = rag_chain("How many trucks has Flagger Force?")
print(result)

In [ ]:
from bs4 import BeautifulSoup
import requests
import pprint

url = 'https://flaggerforce.com/blog'
html_doc = requests.get(url)
pprint.pprint(html_doc.content)

soup = BeautifulSoup(html_doc.content, 'html.parser')


In [ ]:
soup.select('p')

In [ ]:
# 1. split the data into chunks
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter

# urls = ["https://flaggerforce.com/",
#         "https://flaggerforce.com/blog/",
#         "https://flaggerforce.com/about-us/"]

urls = ["https://flaggerforce.com/"]
        
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=100)
doc_splits = text_splitter.split_documents(documents=docs_list)
doc_splits